In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sb 

In [ ]:
data = pd.read_csv('../onlinefraud.csv')
data.head()

In [ ]:
data.info()

In [ ]:
data.shape

In [ ]:
data.iloc[0:1]

In [ ]:
data.isna().sum()

In [ ]:
data.duplicated().sum()

In [ ]:
sb.boxplot(data.oldbalanceOrg)
plt.show()

In [ ]:
data.info()

In [ ]:
sb.boxplot(data['newbalanceDest'])
plt.show()

In [ ]:
data.corr()

In [ ]:
plt.scatter(data.oldbalanceOrg, data.newbalanceDest)
plt.show()

In [ ]:
Q1 = np.percentile(data['amount'], 25,
                   interpolation = 'midpoint')
 
Q3 = np.percentile(data['amount'], 75,
                   interpolation = 'midpoint')
IQR = Q3 - Q1

In [ ]:
IQR

In [ ]:
upper = Q3 +1.5*IQR
lower = Q1 - 1.5*IQR

In [ ]:
upper = np.where(data['amount'] >= (Q3+1.5*IQR))
lower = np.where(data['amount'] <= (Q1-1.5*IQR))

In [ ]:
data.drop(upper[0], inplace = True)
data.drop(lower[0], inplace = True)

In [ ]:
data.step

In [ ]:
data.columns

In [ ]:
data.type

In [ ]:
data.type.unique()

In [ ]:
data.type.value_counts()

In [ ]:
sb.barplot(x = data.type.value_counts().index, y = data.type.value_counts().values)
plt.show()

In [ ]:
data.describe()

In [ ]:
data.amount

In [ ]:
sb.histplot(data.amount)
plt.xlabel('the amount of money')
plt.show()

In [ ]:
data.nlargest(10, 'amount')

In [ ]:
x = data.nlargest(10, 'amount').type
  
def unique(li):
    value_counts = []
    for i in li:
        c = li.count(i)
        value_counts.append((i,c))
    return value_counts

In [ ]:
x

In [ ]:
from collections import Counter
Counter(x)

In [ ]:
data.nlargest(10, 'amount').isFraud

In [ ]:
Counter(data.nlargest(100, 'amount').type)

In [ ]:
x = data.nlargest(100, 'amount').isFraud
counter = 0
if 1 in x.values:
    counter +=1
counter

In [ ]:
plt.hist2d(data= data, x = 'step', y = 'amount', cmin = 0.5,
    cmap = 'viridis_r' )
plt.colorbar()
plt.xlabel('step')
plt.ylabel('the amount')
plt.show()

In [ ]:
step_amount_avg = data[data.amount >= 500000]['step']
plt.hist(step_amount_avg, color = 'black')
plt.axvline(step_amount_avg.mean(), color = 'red')
plt.axvline(step_amount_avg.median(), color = 'green')
plt.xlabel('steps')
plt.show()

In [ ]:
import scipy.stats as st
st.t.interval(alpha=0.90, df=len(step_amount_avg)-1,
              loc=np.mean(step_amount_avg),
              scale=st.sem(step_amount_avg))

In [ ]:
data.info()

In [ ]:
print(f'the number of unique values of nameOrig is : {round((len(data.nameOrig.unique()) * 100) / data.shape[0], 2)} %')

In [ ]:
data[data.nameOrig == 'C1065307291']

In [ ]:
Messi = data.nameOrig.mode()

In [ ]:
original_name = []
for i in Messi.values:
    original_name.append((i,data[data.nameOrig == i].shape[0]))
for j in original_name:
    print(j)

In [ ]:
data.nameOrig = data.nameOrig.str.replace('C','').astype('int')
data.rename(columns = {'nameOrig': 'client_id'}, inplace = True)

In [ ]:
data.client_id

In [ ]:
data.oldbalanceOrg

In [ ]:
data.oldbalanceOrg.describe()

In [ ]:
plt.figure(figsize = [10,7])
plt.hist(data.oldbalanceOrg, bins = 30)
plt.title('the old balance distribution')
plt.show()

In [ ]:
plt.figure(figsize = [10,7])
plt.plot(data.oldbalanceOrg)
plt.show()

In [ ]:
data.nlargest(20, 'oldbalanceOrg')

In [ ]:
plt.figure(figsize = [10,7])
sb.barplot(x = data.nlargest(10, 'oldbalanceOrg').client_id, y = data.nlargest(10, 'oldbalanceOrg').oldbalanceOrg)
plt.xticks(rotation = 17.5)
plt.title('the largest 10 balances before transaction')
plt.xlabel('Client Id')
plt.ylabel('The Balance')
plt.show()

In [ ]:
data.newbalanceOrig

In [ ]:
data.newbalanceOrig.describe()

In [ ]:
plt.figure(figsize = [10,7])
plt.hist(data.newbalanceOrig, bins = 30)
plt.title('the  balance after transaction distribution')
plt.show()

In [ ]:
plt.figure(figsize = [10,7])
plt.plot(data.newbalanceOrig)
plt.show()

In [ ]:
data.nlargest(20, 'newbalanceOrig')

In [ ]:
plt.figure(figsize = [10,7])
sb.barplot(x = data.nlargest(10, 'newbalanceOrig').client_id, y = data.nlargest(10, 'newbalanceOrig').newbalanceOrig)
plt.xticks(rotation = 17.5)
plt.title('the largest 10 balances after transaction')
plt.xlabel('Client Id')
plt.ylabel('The Balance')
plt.show()

In [ ]:
num_of_samples = 10 / data.shape[0]

In [ ]:
data.shape[0]

In [ ]:
samp1 = data.sample(n = 600000)

In [ ]:
samp1

In [ ]:
# Generate some random data
x = np.random.normal(0, 1, size=100)
y = np.random.normal(0, 1, size=100)

# Create a scatter plot with jitter
plt.scatter(x=x, y=y, alpha=0.5, edgecolors='none', s=40, c='blue', linewidth=0.5, rasterized=True)
plt.xlabel('X Axis')
plt.ylabel('Y Axis')
plt.title('Scatter plot with jitter')
plt.show()

In [ ]:
data[['oldbalanceOrg', 'newbalanceOrig']].corr()

In [ ]:
data['balance_diff'] = data.oldbalanceOrg - data.newbalanceOrig

In [ ]:
data.balance_diff.describe()

In [ ]:
plt.figure(figsize = [10,7])
plt.hist(data.balance_diff, bins=50)
plt.title('the  balance after transaction distribution')
plt.show()

In [ ]:
plt.figure(figsize=[10,7])
plt.plot(data.balance_diff)
plt.title('the balance rate of change')
plt.show()

In [ ]:
data.head()


In [ ]:
data.nameDest.describe()

In [ ]:
data.nameDest.mode()

In [ ]:
# get top 10 most frequent client distination id
n = 10
data['nameDest'].value_counts()[:n]

In [ ]:
plt.figure(figsize = [10, 7])
sb.barplot(x = data['nameDest'].value_counts()[:n].index, y = data['nameDest'].value_counts()[:n].values)
plt.xticks(rotation = 20)
plt.title('the most 10 frequent distination clients')
plt.show()

In [ ]:
data[data.nameDest == 'C1286084959']

In [ ]:
totti = data[data.nameDest == 'C1286084959']
totti.describe()

In [ ]:
totti[totti.newbalanceDest == totti.newbalanceDest.max()]

In [ ]:
totti[(totti.isFraud == 1) | (totti.isFlaggedFraud == 1)]

In [ ]:
data[data.isFraud == 1]

In [ ]:
data[data.isFlaggedFraud == 1]

In [ ]:
# get top 10 most frequent fraud distination clients
x = data[data.isFraud == 1]
n = 10
Ramos = x['nameDest'].value_counts()[:n]
Ramos

In [ ]:
for i in Ramos.index:
    if i in data['nameDest'].value_counts()[:n].index:
        print(i)

In [ ]:
data.nameDest = data.nameDest.replace('C','')
data.nameDest = data.nameDest.replace('M','')
data.rename(columns = {'nameDest' : 'distination_client_id'}, inplace = True)

In [ ]:
data.isFraud.value_counts()

In [ ]:
plt.pie(data.isFraud.value_counts(),
        autopct='%1.1f%%', pctdistance=0.85)
centre_circle = plt.Circle((0, 0), 0.70, fc='white')
fig = plt.gcf()
fig.gca().add_artist(centre_circle)
plt.show()

In [ ]:
# class count
class_count_0, class_count_1 = data['isFraud'].value_counts()

# Separate class
class_0 = data[data['isFraud'] == 0]
class_1 = data[data['isFraud'] == 1]# print the shape of the class
print('class 0:', class_0.shape)
print('class 1:', class_1.shape)

In [ ]:
class_0_under = class_0.sample(class_count_1)

balancing_data = pd.concat([class_0_under, class_1], axis=0)

print("total class of 1 and0:",balancing_data['isFraud'].value_counts())# plot the count after under-sampeling
balancing_data['isFraud'].value_counts().plot(kind='bar', title='count (target)')

In [ ]:
balancing_data

In [ ]:
balancing_data.info()

In [ ]:
balancing_data.type.unique()

In [ ]:
balancing_data["type"] = balancing_data["type"].map({"CASH_OUT": 1, "PAYMENT": 2, "CASH_IN": 3, "TRANSFER": 4, "DEBIT": 5})

In [ ]:
balancing_data.type.dtype

In [ ]:
balancing_data.columns

In [ ]:
X = balancing_data[['type', 'amount', 'oldbalanceOrg', 'newbalanceOrig', 'balance_diff']].values
y = balancing_data.loc[:,'isFraud'].values

In [ ]:
x

In [ ]:
y

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
X_train.shape,X_test.shape

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler= StandardScaler()
scaler.fit(X_train)
print(scaler.mean_)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
from sklearn import preprocessing
from sklearn import utils

#convert y values to categorical values
lab = preprocessing.LabelEncoder()
y_train = lab.fit_transform(y_train)
y_test = lab.fit_transform(y_test)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, fbeta_score

In [ ]:
models = {
    "LR": LogisticRegression(),
    "KNN": KNeighborsClassifier(),
    "DT": DecisionTreeClassifier(),
    "RF": RandomForestClassifier(),
    "XGB": XGBClassifier(),
    "Naive Bayes": GaussianNB(),
    "SVC": SVC()

}

In [ ]:
for name, model in models.items():
    print(f'Training Model {name} \n--------------')
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(f'Training Accuracy: {accuracy_score(y_train, model.predict(X_train))}')
    print(f'Testing Accuracy: {accuracy_score(y_test, y_pred)}')
    print(f'Testing Confusion Matrix: \n{confusion_matrix(y_test, y_pred)}')
    print("Recall Score : ",recall_score(y_test, y_pred, pos_label='positive', average='micro'))    
    print("Precision Score : ",precision_score(y_test, y_pred,  pos_label='positive',average='micro'))     
    print(f"Testing F-1:", f1_score(y_test, y_pred, pos_label='positive', average='micro' )        )
    print(f"Testing F-Beta:", fbeta_score(y_test, y_pred, beta=0.5, pos_label='positive', average='micro'))
    print('-'*30)

In [ ]:
model = XGBClassifier()
model.fit(X_train, y_train)
print(model)

In [ ]:
# make predictions for test data
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]

In [ ]:
# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

In [ ]:
rounded_y = np.round(y_pred)
unique_val = pd.DataFrame(y_test)
unique_val[0].unique()

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix
y_unique = unique_val[0].unique()
mcm = multilabel_confusion_matrix(y_test, y_pred, labels = y_unique)
mcm

In [ ]:
data.head()

In [ ]:
if  model.predict([[2,9839.64,170136.0,160296.36,9839.64]])[0] == 0:
    print('not Fraud')
else: print('is Fraud')